<a href="https://colab.research.google.com/github/astrfo/AutonomousOptimalExplorationThroughSatisficing/blob/main/AutonomousOptimalExplorationThroughSatisficing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 満足化に通じた最適な自律的探索

## 2. 多腕バンディット問題

*   定義
    *   基準値: $ R \quad (\aleph)$
    *   報酬： $ r \quad (1 \ or \ 0)$
    *   確率： $ p $
    *   試行量：  $ n(a_i) $
    *   経験期待値： $ V(a_i) $
    *   学習率： $ \alpha $
    *   選択肢の数: $ K $
    *   そのステップで選択された腕： $ a^{select} $
    *   総試行量： $ N $

*   初期値
    *   $ n(a_i) = \epsilon \quad $ (微小)
    *   $ V(a_i) = 0.5 $

*   更新方法
    *   $ \alpha \leftarrow \frac{1}{1 + n(a^{select})} $
    *   $ V(a^{select}) \leftarrow (1 - \alpha) \times V(a^{select}) \ + \  \alpha r $
    *   $ n(a^{select}) \leftarrow n(a^{select}) + 1 $
    *   $ N = \sum_{k=1}^{K} n(a_k) $

$ $

## 3. RSの定義

$$
\begin{eqnarray}
    RS(a_i) &= \frac{n(a_i)}{N}(V(a_i) - R) \quad \cdots (7) \\
    a^{select} &\leftarrow {argmax}_{a_k}(RS(a_k)) \quad \cdots (8) \\
\end{eqnarray}
$$

## 4. 非満足状況 (51:00~)

全ての行動 $a$ の行動期待値 $V(a_i)$ が $V(a_i) < R$ を満たすとき、非満足状況であると定義する。 \\
RS の定義では、試行量とその総和の比率 $\rho(a_i) = \frac{n(a_i)}{N}$ が高ければ高いほど RS 価値関数が算出する評価値は低くなっていく。 \\
2つの選択肢 $(a, b)$ があると仮定して、非満足状況では $a$ を試行するほど、$a$ の RS 評価値が低くなり、$b$ の RS 評価値は高くなる。その後 $b$ を試行すると $b$ の RS 評価値は低くなっていくので、$a$ の RS 評価値が高くなる。(まさに足の引っ張り合い) \\
基準値 $R$ よりも低い $c$ と、高い $d$ の2つの選択肢 $(c, d)$ があると仮定して、$c$ は基準値 $R$ よりも低く、前記の通り、引けば引くほど $RS(c)$ はマイナスになっていくが、$RS(d)$ は引けば引くほど高くなるので選択肢 $d$ がより一層選ばれやすくなる。 \\
総試行量 $N$ が大きくなると RS 評価値はあまり変化しなくなる。そして**いずれ均衡点 $-Z$ に落ち着く。**(これはコード書いて確かめたい) \\
総試行量 $N$ が大きい際の試行量とその総和の比率 $\rho(a_i) = \frac{n(a_i)}{N}$ を逆算することができる。($Z$ が分かるので) \\

$$
RS(a_i) = -Z \\
\rho(a_i) = \frac{n(a_i)}{N} = \frac{Z}{R - V(a_i)} \quad \cdots (10)\\
$$

**(10)式は1番重要らしい。(01:02:35~)** \\
(10)にて、満足する(基準値 $R$ を超える)選択肢が存在するなら 1。 \\
また $Z$ は以下の式で求まる。
$$
Z = \frac{1}{ \sum_{k=1}^{K} \frac{1}{R - V(a_k)} }
$$
RS 均衡値 $-Z$ が基準値 $R$ と経験期待値 $V(a_k)$ から定義できる。 \\

### 4.1 非満足状況下で regret を最適化する基準値

"非満足下の RS 均衡"によって greedy な選択肢　$a_G$ とそれ以外 $a_j$ の RS 評価値が等しくなる。($-Z$ に帰着する) \\

$$
RS(a_G) = RS(a_j) \quad \cdots (15) \\
\begin{eqnarray}
    n(a_G) \times (V(a_G) - R) &= n(a_j) \times (V(a_j) - R) \quad \cdots (16) \\
    n(a_j) &= n(a_G) \frac{(R - V(a_G))}{(R - V(a_j))} \quad \cdots (17) \\
    R &= V(a_G) \frac{ 1 - \frac{V(a_j)}{V(a_G)} \frac{n(a_j)}{n(a_G)} }{ 1 - \frac{n(a_j)}{n(a_G)} } \quad \cdots (18) \\
\end{eqnarray}
$$

ここで $\frac{V(a_j)}{V(a_G)}$ は $\frac{V(a_j)}{V(a_G)} \leq 1$ を満たす。($V(a_G)$ が greedy、非負なので) \\
(15)に RS 定義式を当てると(16)になるし、(16)を式変形すると(17)になる。(17)を $R$ についてまとめると(18)になる。(確認済み) \\
理想的な$\frac{n(a_j)}{n(a_G)}$が分かれば、良い感じの基準値 $R$ が定義できる。 \\
ここでは greedy な選択肢 $a_G$ とそれ以外 $a_j$ の潜在比率 $\mu = \frac{n(a_j)}{n(a_G)}$ を最適化することを目標にする。 \\
<!-- 最適な比率 $\mu^*$ を Chernoff-Hoeffding bound から推定する。(Thompson やら UCB の基となった？) \\ -->
ここで、最適比率 $\mu^*$ は必ずしも $\mu^*\leq1$ になると限らないと思われる(多分)。非満足状態下では、最良の選択肢 $a_G$ を引いても RS 評価値がマイナスになるので選び続けるということがないため、$n(a_j)  > n(a_G)$ はあり得る事象だと思う。個人的には $-Z$ に帰着するため大体同じ回数選ばれるのでは？と思うので　$\mu^* = 1$　かな思っている。 \\
ここで、総試行量 $N$ で $n^*(a_i)$ 割ると最適な試行割合　$\rho^*_i$ が出てくる。
$$
\mu^*_j = \frac{n^*(a_j)}{N} \frac{N}{n^*(a_G)} = \frac{\rho^*_j}{\rho^*_G}
$$
ここでは、$\rho^*_i$ が $V^*(a_i) \geq V^*(a_G)$ となる確率と一致すると定義する。そして Chernoff-Hoeffding bound から以下の式を定義する。

$$
Pr(V(a_j) \geq p_i + \epsilon) \leq exp(-n(a_j) D_{KL}((p_i + \epsilon) \ || \ p_i)) \quad \cdots (21) \\
\begin{align}
    \rho^*_i
    &= \frac{n^*(a_i)}{N} \\
    &= Pr(V^*(a_i) \geq V^*(a_G)) \\
    &= exp(-n(a_i) D_{KL}(V^*(a_i) \ || \ V^*(a_G))) \quad \cdots (22) \\
\end{align}
$$
カルバックライブラー情報量について、二つの確率分布の差異を表す。$V^*(a_G)$のような理論値と$V^*(a_i)$とのカルバックライブラー情報量を計算する。 \\
$$
D_{KL}((p_i + \epsilon) \ || \ p_i)) = \sum_{i}(p_i + \epsilon) \log \frac{p_i + \epsilon}{p_i} \\
D_{KL}(V^*(a_i) \ || \ V^*(a_G)) = \sum_{i} V^*(a_i) \log \frac{V^*(a_i)}{V^*(a_G)}
$$

ここで、ある選択肢が $a_G$ だった場合、
$$
\begin{align}
    \rho^*_G
    &= \frac{n^*(a_G)}{N} \\
    &= Pr(V^*(a_G) \geq V^*(a_G)) \\
    &= exp(-n(a_G) D_{KL}(V^*(a_G) \ || \ V^*(a_G))) \\
    &= 1 \quad \cdots (23) \\
\end{align}
$$
$\rho^*_k$ は目標の試行割合なので、$\sum_{}\rho^*_k = 1$ を満たさなければならないというわけではない。 \\
近似的に最適な潜在選択比率の推定値 $\mu^{CH}_j \fallingdotseq \mu^*_j$ は以下の定義になる。
$$
\begin{align}
    \mu^{CH}_j
    &= \frac{n^*(a_j)}{N} \frac{N}{n^*(a_G)} = \frac{\rho^*_j}{\rho^*_G} = \rho^*_j \\
    &= exp(-n(a_j) D_{KL}(V^*(a_j) \ || \ V^*(a_G))) \quad \cdots (24) \\
    R^{CH} &= max(V(a_G) \frac{1 - \frac{V(a_j)}{V(a_G)} \mu^{CH}_j}{1 - \mu^{CH}_j}) \quad \cdots (25) \\
\end{align}
$$
(25)は(18)より求められる。 \\
$\mu^{CH}_j$ を用いて算出された基準値 $R$ を**「非満足基準値 $R^{CH}$」** とし、それを用いた RS 価値関数を **RS-CH** と呼ぶ。

## 5. RS-CHの多腕対応

RS-CH は、2本腕($a_j$ と $a_G$)バンディット問題しか使用できない。 \\

### 5.1 3本腕バンディット問題以上での非最適性

総試行量 $N$ が十分大きい場合、試行割合 $\rho^*(a_i)$ は(26)になる。
$$
\begin{align}
    \rho^*(a_i) &= exp(-n(a_i) D_{KL}(V^*(a_i) \ || \ V^*(a_i))) \quad \cdots (26) \\ 
    \frac{\rho^*(a_i)}{\rho^*(a_j)} &= \frac{exp(-n(a_i) D_{KL}(V^*(a_i) \ || \ V^*(a_i)))}{exp(-n(a_j) D_{KL}(V^*(a_j) \ || \ V^*(a_j)))} \quad \cdots (27) \\
\end{align}
$$
RS 価値関数に対して greedy な選択をし続けた最終的な試行割合 $\rho^{RS}$ (RS 目的試行割合) は(28)である。
$$
\begin{align}
    \rho^{RS}(a_i) &= \frac{Z}{R - V(a_i)} \quad \cdots (28) \\
    \frac{\rho^{RS}(a_i)}{\rho^{RS}(a_j)} &= \frac{R - V(a_j)}{R - V(a_i)} \quad \cdots (29) \\
\end{align}
$$
(29)が $N \rightarrow \infty$ の時に(27)を満たす基準値 $R$ が存在しない限り、RS は3本腕バンディット問題に最適ではない。

### 5.2 3本腕以上のバンディット問題への対応

RS 価値関数の場合、全ての選択肢に対して基準値 $R$ を用いて後悔の最小化は不可能。 \\
この研究では、greedy な選択肢 $a_G$ とそれ以外 $a_j$ を比較する。(2本腕バンディットのように) \\
(25)と RS 価値関数((7), (8)らへん)を計算する。それで greedy な $a_G$ と 非 greedy な $a_j$ を比較して RS 評価値が最も高い選択肢を選ぶ。もし $a_G$ の RS 評価値を上回る選択肢が複数存在していた場合は $a_G$ との潜在的選択比率 $\mu^*_j$ が最も高い選択肢を選ぶ。また、最も高い潜在的選択比率 $\mu^*_j$ も複数あった場合は、 $R^{CH}$ が最も高い選択肢を選ぶ。

## 6. 実験と結果

基準値 $R^{OPT}$ は理想的な基準値のことで、それが既知の場合が RS-OPT の定義。理想的な基準値 $R^{OPT}$ は選択肢 $a_k$ を真の報酬出現確率 $p_k$ に対して同じ値を排除して高い順に並べた時の選択肢 $q_m$ に対して以下の定義とする。
$$
RS^{OPT} = \frac{q_1 + q_2}{2}
$$
要するに、1番当たりを引く腕と2番目の腕に着目した式だと思う。 \\
評価指標は regret。
$$
regret = \sum_{t=0}^{T}(q_1 - p^{select}_t)
$$

## 7. 今後の展望

RS-CH と従来の RS の違いは基準値 R の定義のみのため、色々組み合わせて非定常環境(確率等が変化する)でも用いることができる可能性がある。 \\
それ以外にも、$R^{CH}$ の更新に使うステップ数をずらすだけで非定常環境に強くなる可能性が考えられるらしいです。 \\
RS-CH は２値の報酬(当たり1, ハズレ0)でのバンディット問題におけるカルバックライブラー情報量での定義のため、報酬の分布が分かればベルヌーイバンディット問題以外にも適用できる。


### 7.1 非満足状況下での最適基準値と理想基準値との関係



## 参考文献

1. [learnlatex] https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/learnlatex.ipynb#scrollTo=jZ4nUZbeqf2I

2. [LaTeX数学記号コマンド] https://medemanabu.net/latex/operators/

3. [確率的バンディット問題] https://www.slideshare.net/jkomiyama/ss-34796421

4. [Kullback-Leibler] https://ja.wikipedia.org/wiki/%E3%82%AB%E3%83%AB%E3%83%90%E3%83%83%E3%82%AF%E3%83%BB%E3%83%A9%E3%82%A4%E3%83%96%E3%83%A9%E3%83%BC%E6%83%85%E5%A0%B1%E9%87%8F